# Install libraries 


In [1]:
import sys, os, json
import mne, sklearn, wandb
import numpy as np
import pandas as pd

from scipy.interpolate import interp1d
from nilearn import datasets, image, masking, plotting
from nilearn.input_data import NiftiLabelsMasker


# animation part
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
# from celluloid import Camera   # it is convinient method to animate
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation


## torch libraries 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader, Subset

from pytorch_model_summary import summary

%load_ext autoreload
%autoreload 2
sys.path.insert(1, os.path.realpath(os.path.pardir))

from utils import get_datasets
from utils import preproc
from utils import torch_dataset
from utils import train_utils
from utils import inference
from utils.models_arch import autoencoder_new_Artur

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


# Set all hyperparameters
- Cuda and GPU.
- Parameters of dataset. 
- random seed( if necessary). 


In [3]:
# import random

# torch.manual_seed(0)
# random.seed(0)  # python operation seed
# np.random.seed(0)

# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True

print(torch.cuda.is_available(), torch.cuda.device_count())
torch.cuda.set_device(0)

True 4


In [4]:
config = dict(  
                dataset_name = 'CWL_raw', # CWL
                patients = ['trio1'], # work only with one patient
                fps = 1000,
                new_fps = 100, 
                crop_start = 5,
                freqs = [-1], 
    
                n_channels = 30, # 63 
                n_roi = 8,
                
                bold_delay = 6,
                to_many = True,
                random_subsample = True,
                sample_per_epoch = 512, 
                WINDOW_SIZE = 2048,
                    
                optimizer='adamW',
                lr=3e-4,
                weight_decay=3e-4, 
                batch_size=16, 
                    
                mse_weight = 1,
                corr_weight = 0,
                
                preproc_type = 'dB_log',
                loss_function = 'corr', 
                model_type = 'Best_AE_Artur_Multi_Head'
                )


hp_autoencoder = dict(n_electrodes=config['n_channels'],
                      n_freqs = len(config['freqs']),
                      n_channels_out = config['n_roi'],

                     channels = [128, 128, 128, 128], 
                     kernel_sizes=[5, 5, 3],
                     strides=[8, 8, 4], 
                     dilation=[1, 1, 1], 
                     decoder_reduce=4, 
                     hidden_channels = 16,
                     )


config = {**hp_autoencoder, **config}

params_train = {'batch_size': config['batch_size'],
                'shuffle': True,
                'num_workers': 0}

params_val = {'batch_size': config['batch_size'],
              'shuffle': False}

# Upload preprocessed dataset from np files. 
It should accelerate speed of experiments.

In [11]:
# with open("../data/interim/labels_roi_17.json", 'r') as f:
#     labels_roi_17 = json.load(f)
    
labels_roi = ['Left Pallidum',
                 'Left Caudate',
                 'Left Putamen',
                 'Left Accumbens',

                 'Right Pallidum',
                 'Right Caudate',
                 'Right Putamen',
                 'Right Accumbens']

## Download CWL dataset

You can save it into numpy files and the train only on such files. 

In [30]:
path_to_dataset = '../data/eyes_open_closed_dataset/'
dataset_name = 'CWL'
remove_confounds = True

for patient in config['patients']:
    df_eeg_cwl_raw, df_fmri_cwl_raw, labels_roi_17 = get_datasets.download_cwl_dataset(patient, path_to_dataset, 
                                                                        remove_confounds=remove_confounds,
                                                                        verbose=True)
    df_eeg_cwl, df_fmri_cwl, fps = get_datasets.interpolate_df_eeg_fmri(df_eeg_cwl_raw, df_fmri_cwl_raw)
    
    
    config['fps'] = fps
    print('Original FPS', config['fps']) 
    
    # delete time columns. 
    # reshape [time, ch] -> [ch, time]
    eeg_np = df_eeg_cwl.drop(['time'], axis=1).to_numpy().T
    fmri_np = df_fmri_cwl.drop(['time'], axis=1).to_numpy().T
    
    
    eeg_np = preproc.low_level_preproc_eeg(eeg_np, fps)
    
    data = {'eeg': eeg_np, 
            'fmri': fmri_np}
    
    # np.savez(f'../data/preproc/{config["dataset_name"]}/{patient}_{config["fps"]}_filtered_data', 
    #          eeg=eeg_np, fmri=fmri_np)

ALL path:  ../data/eyes_open_closed_dataset/trio1/CWL_Data/eeg/in-scan/trio1_mrcorrected_eoec_in-scan_hpump-off.set ../data/eyes_open_closed_dataset/trio1/CWL_Data/mri/epi_normalized/rwatrio1_eoec_in-scan_hpump-off.nii ../data/eyes_open_closed_dataset/trio1/CWL_Data/mri/epi_motionparams/rp_atrio1_eoec_in-scan_hpump-off.txt


/home/user/common/koval_alvi/eeg-fmri-project/utils/get_datasets.py:215: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(eeg_path_set_file)
/home/user/common/koval_alvi/eeg-fmri-project/utils/get_datasets.py:215: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_path_set_file)
/home/user/common/koval_alvi/eeg-fmri-project/utils/get_datasets.py:173: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  motion_confound = pd.read_csv(motion_params_path, sep = '  ', header=None)


Dimension of our EEG data:  (303601, 31)
Dimension of our fMRi data:  (61, 72, 61, 146)
Dimension of our fMRi Roi data:  (143, 18)
fMRI info :  1.95
RoI:  ['Left Lateral Ventricle', 'Left Thalamus', 'Left Caudate', 'Left Putamen', 'Left Pallidum', 'Brain-Stem', 'Left Hippocampus', 'Left Amygdala', 'Left Accumbens', 'Right Lateral Ventricle', 'Right Thalamus', 'Right Caudate', 'Right Putamen', 'Right Pallidum', 'Right Hippocampus', 'Right Amygdala', 'Right Accumbens', 'time']
Original FPS 1000
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 sec)



/home/user/common/koval_alvi/eeg-fmri-project/utils/get_datasets.py:261: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  return df_eeg, df_fmri, df_fmri.drop(['time'], 1).columns.to_list()


Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (3.301 sec)



In [29]:
df_eeg_cwl_raw['time'], df_fmri_cwl_raw['time']

(0              0
 1              1
 2              2
 3              3
 4              4
            ...  
 303596    303596
 303597    303597
 303598    303598
 303599    303599
 303600    303600
 Name: time, Length: 303601, dtype: int64,
 0       23322.0
 1       25272.0
 2       27222.0
 3       29172.0
 4       31122.0
          ...   
 138    292423.0
 139    294373.0
 140    296323.0
 141    298273.0
 142    300223.0
 Name: time, Length: 143, dtype: float64)

## Preprocessing of datasets

- get only useful fmri ROI 
- Cut starting points 
- Normalize 
- Splitting on train/test
- downsampling to new_fps 
- Shifting between EEG and FMRI 
- Dataset and DataLoader 

In [18]:

eeg, fmri = data['eeg'], data['fmri']
df = pd.DataFrame(data = fmri.T, columns=labels_roi_17)
df_filter = df[labels_roi]
fmri = df_filter.to_numpy().T

# crop start
train_crop = config['crop_start']*config['fps']
eeg, fmri = eeg[..., train_crop:], fmri[..., train_crop:]

# normalize 
eeg = eeg / np.std(eeg)
fmri, fmri_means_stds = preproc.normalize_data(fmri)

# train/test split
test_time = int(60*config['fps'])
train_dataset_prep = (eeg[..., :-test_time], fmri[..., :-test_time])
test_dataset_prep = (eeg[..., -test_time:], fmri[..., -test_time:])


ds_factor = config['fps']/config['new_fps']
train_dataset_prep = preproc.downsample_dataset(train_dataset_prep, factor = ds_factor)
test_dataset_prep = preproc.downsample_dataset(test_dataset_prep, factor = ds_factor)





# apply time dealy corrected
train_dataset_prep = preproc.bold_time_delay_align(train_dataset_prep, 
                                                   config['new_fps'],
                                                   config['bold_delay'])
test_dataset_prep = preproc.bold_time_delay_align(test_dataset_prep, 
                                                  config['new_fps'],
                                                  config['bold_delay'])


print('Size of train dataset:', train_dataset_prep[0].shape, train_dataset_prep[1].shape)
print('Size of test dataset:', test_dataset_prep[0].shape, test_dataset_prep[1].shape)

# torch dataset creation 
torch_dataset_train = torch_dataset.CreateDataset_eeg_fmri(train_dataset_prep, 
                                                            random_sample=config['random_subsample'], 
                                                            sample_per_epoch=config['sample_per_epoch'], 
                                                            to_many=config['to_many'], 
                                                            window_size = config['WINDOW_SIZE'])

torch_dataset_test = torch_dataset.CreateDataset_eeg_fmri(test_dataset_prep, 
                                                            random_sample=False, 
                                                            sample_per_epoch=None, 
                                                            to_many=config['to_many'], 
                                                            window_size = config['WINDOW_SIZE'])
print('Size of test dataset:', len(torch_dataset_test))
# because you do not have strid for val data. 
torch_dataset_test = Subset(torch_dataset_test, np.arange(len(torch_dataset_test))[::100])

# init dataloaders for training
train_loader = torch.utils.data.DataLoader(torch_dataset_train, **params_train)
val_loader = torch.utils.data.DataLoader(torch_dataset_test, **params_val)




Size of train dataset: (30, 20590) (8, 20590)
Size of test dataset: (30, 5400) (8, 5400)
Size of test dataset: 3351


# Init Model, Loss, optimizers

In [15]:
model = autoencoder_new_Artur.AutoEncoder1D_Artur_MultiHead(hp_autoencoder)

print(summary(model, torch.zeros(4, config['n_channels'],
                                 config['WINDOW_SIZE']), show_input=False))


-----------------------------------------------------------------------------
            Layer (type)        Output Shape         Param #     Tr. Param #
   AutoEncoder1D_Artur-1        [4, 1, 2048]         245,057         245,057
   AutoEncoder1D_Artur-2        [4, 1, 2048]         245,057         245,057
   AutoEncoder1D_Artur-3        [4, 1, 2048]         245,057         245,057
   AutoEncoder1D_Artur-4        [4, 1, 2048]         245,057         245,057
   AutoEncoder1D_Artur-5        [4, 1, 2048]         245,057         245,057
   AutoEncoder1D_Artur-6        [4, 1, 2048]         245,057         245,057
   AutoEncoder1D_Artur-7        [4, 1, 2048]         245,057         245,057
   AutoEncoder1D_Artur-8        [4, 1, 2048]         245,057         245,057
Total params: 1,960,456
Trainable params: 1,960,456
Non-trainable params: 0
-----------------------------------------------------------------------------


# Model training

In [17]:
n_runs = 3

for i in range(n_runs):
    
    model = autoencoder_new_Artur.AutoEncoder1D_Artur_MultiHead(hp_autoencoder)

    loss_func = train_utils.make_complex_loss_function(mse_weight = config['mse_weight'], 
                                                       corr_weight = config['corr_weight'],
                                                       manifold_weight = 0,
                                                       bound=1)
    train_step = train_utils.train_step

    optimizer = optim.AdamW(model.parameters(), 
                       lr=config['lr'], 
                       weight_decay=config['weight_decay'])
    
    
    parameters = {
        'EPOCHS': 1500,
        'model': model, 
        'train_loader': train_loader, 
        'val_loader': val_loader, 
        'loss_function': loss_func,
        'train_step': train_step,
        'optimizer': optimizer, 
        'device': 'cpu', 
        'raw_test_data': test_dataset_prep,
        'show_info': 20, 
        'num_losses': 5,
        'labels': labels_roi,
        'inference_function': inference.model_inference_function, 
        'to_many': config['to_many']
    }



    path_to_save_wandb = 'common/koval_alvi/Checkpoints/wandb_brain'
    
    
    with wandb.init(project="eeg_fmri", config=config, save_code=True):
        
        wandb.define_metric("val/corr_mean", summary="max")

        if i == 0: 
            exp_name = wandb.run.name
        
        wandb.run.name = exp_name +'_run_' + str(i)
        
        print(config)
        print(parameters['model'])
        print(summary(model, torch.zeros(4, config['n_channels'], config['WINDOW_SIZE']), show_input=False))
        
        model = train_utils.wanb_train_regression(**parameters)
        

{'n_electrodes': 30, 'n_freqs': 1, 'n_channels_out': 8, 'channels': [128, 128, 128, 128], 'kernel_sizes': [5, 5, 3], 'strides': [8, 8, 4], 'dilation': [1, 1, 1], 'decoder_reduce': 4, 'hidden_channels': 16, 'dataset_name': 'CWL_raw', 'patients': ['trio1'], 'fps': 1000, 'new_fps': 100, 'crop_start': 5, 'freqs': [-1], 'n_channels': 30, 'n_roi': 8, 'bold_delay': 6, 'to_many': True, 'random_subsample': True, 'sample_per_epoch': 512, 'WINDOW_SIZE': 2048, 'optimizer': 'adamW', 'lr': 0.0003, 'weight_decay': 0.0003, 'batch_size': 16, 'mse_weight': 1, 'corr_weight': 0, 'preproc_type': 'dB_log', 'loss_function': 'corr', 'model_type': 'Best_AE_Artur_Multi_Head'}
AutoEncoder1D_Artur_MultiHead(
  (models): ModuleList(
    (0): AutoEncoder1D_Artur(
      (artur_block): ArturBlock(
        (unmixing_layer): Conv1d(30, 16, kernel_size=(1,), stride=(1,))
        (unmixed_channels_batchnorm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        (band_pass): Conv1d(16, 

wandb: ERROR Control-C detected -- Run data was not synced

KeyboardInterrupt



# 